In [1]:
import pandas as pd

# 1 -  Fuzzy Matching

Utilizaremos los conceptos de fuzzy matching para mejorar una variable categórica

In [2]:
#Primero instalemos fuzzywuzzy
#!pip install fuzzywuzzy

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Leemos un archivo con información de los departamentos de Uruguay
fuzzy = pd.read_csv("../DataSets/Ejercicio_Fuzzy.csv", encoding = "ISO-8859-1")

f:\Personal\django\CPE\CPEDS\envWindows\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
fuzzy.sample(6)

,Id,Departamento,Valor
70,71,Paysandu,89
67,68,Rivera,35
14,15,Treinta y Tres,49
37,38,Soriano,55
63,64,Canelons,52
66,67,Rivera,47


Tenemos un dataset de ejemplo con un campo "Departamento" ingresado de forma manual, con errores de tipeo

In [6]:
fuzzy['Departamento'].drop_duplicates()

0         Montevideo
1          Canelones
2           San Jose
3            Soriano
4            Durazno
5        Cerro Largo
6           C. Largo
7            S. Jos
8           San Jos
9             Mvdeo.
10           Montev.
11           Colonia
12           durazno
14    Treinta y Tres
15      Treintaytres
16         Rio Negro
18          R. negro
19         Maldonado
21    Trainta y trs
22         maldonado
23             Rocha
24        Tacuarmebo
25       Tamcuaremb¢
26          Canelons
27           Artigas
28             Salto
29            Rivera
32          Paysand£
33          Paysandu
75           Florida
76             Flors
77            Flores
79         Lavalleja
Name: Departamento, dtype: object

De la librería fuzzywuzzy usaremos las funciones:


`ratio(string_1, string_2)`: calcula el ratio de coincidencia entre string_1 y string_2 usando la distancia de Levenshtein

`partial_ratio(string_1, string_2)`: calcula el ratio de coincidencia tomando al string más corto como substring y buscando coincidencias parciales



In [7]:
# Ratio entre strings iguales
fuzz.ratio('durazno','durazno')

100

In [8]:
# Ratio entre strings levemente distintos
fuzz.ratio('Durazno','durazno')

86

In [9]:
# Ratio entre strings muy distintos
fuzz.ratio('durazno','Yo vivo en durazno hace años')

40

In [10]:
# Ratio parcial entre strings muy distintos
fuzz.partial_ratio('durazno','Yo vivo en durazno hace años')

100

Obtenemos una lista con los valores verdaderos de los departamentos.

In [11]:
departamentos = [
    'Montevideo',
    'Treinta y Trés',
    'Colonia',
    'Artigas',
    'Canelones',
    'Cerro Largo',
    'Durazno',
    'Flores',
    'Florida',
    'Lavalleja',
    'Maldonado',
    'Paysandú',
    'Río Negro',
    'Rivera',
    'Rocha',
    'Salto',
    'San José',
    'Soriano',
    'Tacuarembó'
]

Busquemos encontrar el valor más próximo para uno de los valores de nuestro dataset.
En general es un buen procedimiento utilizar las técnicas de manipulación de strings como quitar eliminar espacios en blanco, llevar todo el texto a minuscula, etc. previo a usar las funciones de fuzzy matching funciones.

In [12]:
fuzzy.loc[76,'Departamento']

'Flors'

In [13]:
depto = fuzzy.loc[76,'Departamento'].lower()
depto

'flors'

In [14]:
max_coincidencia = 0
# Recorremos la lista de departamentos
for valor_correcto in departamentos:
  # Llevamos a minuscula
  valor_correcto = valor_correcto.lower()
  # Obtenemos el ratio de coincidencia
  coincidencia = fuzz.ratio(depto, valor_correcto)
  # Si el puntaje de la coincidencia es mayor que la vigente
  if coincidencia > max_coincidencia:
    # Guardar el valor correcto
    correccion = valor_correcto
    # Reemplazar el puntaje
    max_coincidencia = coincidencia

print(f"Para el valor {depto}, el valor correcto es {correccion} con un puntaje de {max_coincidencia}")

Para el valor flors, el valor correcto es flores con un puntaje de 91


In [15]:
for _,row in fuzzy.iterrows():
  depto = row['Departamento'].lower()
  max_coincidencia = 0
  # Recorremos la lista de departamentos
  for valor_correcto in departamentos:
    # Llevamos a minuscula
    valor_correcto = valor_correcto.lower()
    # Obtenemos el ratio de coincidencia
    coincidencia = fuzz.ratio(depto, valor_correcto)
    # Si el puntaje de la coincidencia es mayor que la vigente
    if coincidencia > max_coincidencia:
      # Guardar el valor correcto
      correccion = valor_correcto
      # Reemplazar el puntaje
      max_coincidencia = coincidencia
  if max_coincidencia < 100:
    print(f"Para el valor {depto}, el valor correcto es {correccion} con un puntaje de {max_coincidencia}")

Para el valor san jose, el valor correcto es san josé con un puntaje de 88
Para el valor c. largo, el valor correcto es cerro largo con un puntaje de 74
Para el valor s. jos, el valor correcto es san josé con un puntaje de 67
Para el valor san jos, el valor correcto es san josé con un puntaje de 88
Para el valor mvdeo., el valor correcto es montevideo con un puntaje de 62
Para el valor montev., el valor correcto es montevideo con un puntaje de 71
Para el valor treinta y tres, el valor correcto es treinta y trés con un puntaje de 93
Para el valor treintaytres, el valor correcto es treinta y trés con un puntaje de 85
Para el valor rio negro, el valor correcto es río negro con un puntaje de 89
Para el valor r. negro, el valor correcto es río negro con un puntaje de 82
Para el valor trainta y trs, el valor correcto es treinta y trés con un puntaje de 86
Para el valor tacuarmebo, el valor correcto es tacuarembó con un puntaje de 80
Para el valor tamcuaremb¢, el valor correcto es tacuarem

# 2 - Ejercicios con expresiones regulares

In [16]:
entrevistas_dict = {'nombre':['Pedro', 'Celeste', 'Mariela', 'Jorge', 'Matías'],
 'fecha_nacimiento':['22-03-1994', 'Nací el 15-05-1997', '03-11-91', 'fecha nacimiento: 10-10-1998', '03-01-1999'],
 'titulo_maximo': ['Lic. Administracion', 'Ingeniera en sistemas', 'Licenciada en Economía', 'Ing. industrial', 'Biología'],
 'idiomas':['español', 'español, ingles', 'español', 'español, ingles y portugues', '4']}

In [17]:
entrevistas = pd.DataFrame(entrevistas_dict)

In [18]:
entrevistas

,nombre,fecha_nacimiento,titulo_maximo,idiomas
0,Pedro,22-03-1994,Lic. Administracion,español
1,Celeste,Nací el 15-05-1997,Ingeniera en sistemas,"español, ingles"
2,Mariela,03-11-91,Licenciada en Economía,español
3,Jorge,fecha nacimiento: 10-10-1998,Ing. industrial,"español, ingles y portugues"
4,Matías,03-01-1999,Biología,4


Supongamos que queremos crear una variable limpia con la fecha de nacimiento

In [19]:
# Usamos el método findall 
entrevistas['fecha_nacimiento'].str.findall(pat="\d{2}-\d{2}-\d{2,4}")

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\carlos\AppData\Local\Temp\ipykernel_1112\1126923243.py:2: SyntaxWarning: invalid escape sequence '\d'
  entrevistas['fecha_nacimiento'].str.findall(pat="\d{2}-\d{2}-\d{2,4}")


0    [22-03-1994]
1    [15-05-1997]
2      [03-11-91]
3    [10-10-1998]
4    [03-01-1999]
Name: fecha_nacimiento, dtype: object

Ahora queremos crear una variable que nos indique si la persona estudió ingeniería 

In [20]:
#Usamos el método contains
entrevistas['titulo_maximo'].str.contains(pat='Ing|ingenier')

0    False
1     True
2    False
3     True
4    False
Name: titulo_maximo, dtype: bool

Podemos simplificar aún más la expresión regular

In [21]:
entrevistas['titulo_maximo'].str.contains(pat='I|ing')

0    False
1     True
2    False
3     True
4    False
Name: titulo_maximo, dtype: bool

Ahora queremos analizar la variable de idiomas. En primer lugar queremos controlar que las personas hayan puesto un listado de idiomas y no otro dato

In [22]:
entrevistas['idiomas'].str.contains(pat="\d")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\carlos\AppData\Local\Temp\ipykernel_1112\2084411510.py:1: SyntaxWarning: invalid escape sequence '\d'
  entrevistas['idiomas'].str.contains(pat="\d")


0    False
1    False
2    False
3    False
4     True
Name: idiomas, dtype: bool

Vemos que el último registro tiene un dato numérico, el cual no nos sirve para este análisis que queremos realizar. Procedemos a generar una variable con la lista de idiomas

In [23]:
entrevistas['idiomas'].str.split(pat="[,y]")

0                          [español]
1                 [español,  ingles]
2                          [español]
3    [español,  ingles ,  portugues]
4                                [4]
Name: idiomas, dtype: object

Si quisieramos generar una columna por cada elemento podemos utilizar el argumento `expand=True`

In [24]:
entrevistas['idiomas'].str.split(pat="[,y]", expand=True)

,0,1,2
0,español,None,None
1,español,ingles,None
2,español,None,None
3,español,ingles,portugues
4,4,None,None


# 3 - Entrada y salida de datos

Vimos que existe una gran cantidad de formatos de archivos y pandas brinda métodos para leer y escribir en cada uno de ellos.

Uno de los formatos más utilizados es el csv. Observemos algunos de los parámetros más usuales al leer un archivo

In [25]:
# Leemos el archivo de ejemplo que viene en google colab
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv")
housing_df.sample(6)

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/california_housing_train.csv'

In [ ]:
housing_df.info()

In [ ]:
# Especificamos una columna de indice
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv", index_col='longitude')
housing_df.head(6)

In [ ]:
# Especificamos una fila como nombre de columnas
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv", header=1)
housing_df.sample(6)

In [ ]:
# Indicamos que lea las primeras 1000 filas
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv", nrows=1000)
housing_df.info()

Hemos viste que podemos leer archivos pasando una URL en la función correspondiente. 
Una función muy útil para leer tablas desde ciertos sitios web es `pandas.read_html(io, match)`:

*   **io**: dirección o nombre del archivo
*   **match**: string o regex para encontrar tablas particulares

La función devuelve una lista de DataFrames con todas las tablas en el html.



In [ ]:
? pd.read_html()

Veamos un caso de análisis para la página de Wikipedia de la [Copa Mundial de Fútbol](https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol) 

In [ ]:
#Busquemos ahora tablas dentro de una pagina web de wikipedia
mundial = pd.read_html('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol')
print(f'Total de tablas en la pagina: {len(mundial)}')

In [ ]:
# Veamos la quinta tabla: tabla de goleadores del mundial
mundial[4]

Podemos utilizar el argumento match para obtener un subconjunto de las tablas

In [ ]:
#Traigamos la tabla del balon de oro
tabla_balon_oro = pd.read_html('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol',match='Oro')
print(f'Total de tablas en la pagina: {len(tabla_balon_oro)}')

In [ ]:
df = tabla_balon_oro[0]
df

## 4 - Conexion con una **API**

Veamos un muy breve ejemplo de interacción con una API



In [ ]:
# Cargamos el módulo requests
import requests

In [ ]:
# Vamos a consultar la api de Github de pandas
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
# Realizamos una operacion GET a la url
r = requests.get(url)

In [ ]:
r.json()[0:2]

In [ ]:
# Podemos construir un dataframe con esta respuesta
pd.DataFrame(r.json())